In [1]:
%autosave 20

import pandas as pd
import numpy as np
from geopy.geocoders import ArcGIS

nom = ArcGIS()

# print all the outputs in a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Autosaving every 20 seconds


In [2]:
research = pd.read_excel('/Users/chensumei/Desktop/Ram Paper/FinalSheets/Combined Research(by Region).xlsx', index_col=0)
others = pd.read_excel('/Users/chensumei/Desktop/Ram Paper/FinalSheets/snap_warn_disaster.xlsx', index_col=0)

In [3]:
research.head()

,Search,Search term,Region,State,Year,Month
0,24,calfresh,Los Angeles CA,California,2019,1
1,34,calfresh,Los Angeles CA,California,2019,1
2,26,calfresh,Los Angeles CA,California,2019,1
3,26,calfresh,Los Angeles CA,California,2019,1
4,28,calfresh,Los Angeles CA,California,2019,2


In [4]:
research.Region.unique()

array(['Los Angeles CA', 'Sacramento-Stockton-Modesto CA', 'San Diego CA',
       'San Francisco-Oakland-San Jose CA', 'Austin', 'DFW', 'Houston',
       'SanAntonio'], dtype=object)

In [5]:
others.head()

,County,SNAP_Applications,Year,Month,State,Number of Workers,# of disaster
0,Anderson,405,2019,1,Texas,0,0
1,Anderson,243,2019,2,Texas,0,0
2,Anderson,282,2019,3,Texas,0,0
3,Anderson,274,2019,4,Texas,0,0
4,Anderson,317,2019,5,Texas,0,0


In [6]:
research.isna().sum()

Search         0
Search term    0
Region         0
State          0
Year           0
Month          0
dtype: int64

In [7]:
others.isna().sum()

County               0
SNAP_Applications    0
Year                 0
Month                0
State                0
Number of Workers    0
# of disaster        0
dtype: int64

## Table about County - unique county and its state

In [8]:
# need the county, State
county = others[['County', 'State']]


In [9]:
county_table = county.groupby(['State', 'County']).agg(counts = ('County', 'size'))

In [10]:
county_table.reset_index(inplace=True)

In [11]:
county_table.head()

,State,County,counts
0,California,Alameda,36
1,California,Alpine,36
2,California,Amador,36
3,California,Butte,36
4,California,Cahuilla Indian Reservation,1


In [12]:
county_table.drop(columns=['counts'], inplace=True)

In [13]:
county_table['latitude'] = 0.0
county_table['longitude'] = 0.0

In [14]:
#list(nom.geocode('Wise' + 'county' + 'Texas'))

In [15]:
#county_table[315:]

In [16]:
for idx in range(len(county_table)):
    try:
        add = county_table.iloc[idx, 1] + 'county' + county_table.iloc[idx, 0]
        cur_la_lo = list(nom.geocode(add))[1]
        county_table.iloc[idx, 2] = cur_la_lo[0]
        county_table.iloc[idx, 3] = cur_la_lo[1]
    except:
        print('Something wrong with the idx:' + str(idx))

Something wrong with the idx:4
Something wrong with the idx:17
Something wrong with the idx:29
Something wrong with the idx:50
Something wrong with the idx:197


In [17]:
id = [4, 17, 29, 50, 54, 197]

county_table.iloc[id]
    

,State,County,latitude,longitude
4,California,Cahuilla Indian Reservation,0.000000,0.000000
17,California,La Jolla Indian Reservation,0.000000,0.000000
29,California,Morongo Indian Reservation,0.000000,0.000000
50,California,Soboba Indian Reservation,0.000000,0.000000
54,California,Sutter,39.034675,-121.694846
197,Texas,Kickapoo Reservation,0.000000,0.000000


In [18]:
# fill the empty first
id = [4, 17, 29, 50, 197]

for idx in id:
    try:
        add = county_table.iloc[idx, 1]  + county_table.iloc[idx, 0]
        cur_la_lo = list(nom.geocode(add))[1]
        county_table.iloc[idx, 2] = cur_la_lo[0]
        county_table.iloc[idx, 3] = cur_la_lo[1]
    except:
        print('Something wrong with the idx:' + str(idx))


In [19]:
# then drop 54 and 295

#county_table.drop(index=[54, 295], inplace=True)

In [20]:
county_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   State      320 non-null    object 
 1   County     320 non-null    object 
 2   latitude   320 non-null    float64
 3   longitude  320 non-null    float64
dtypes: float64(2), object(2)
memory usage: 10.1+ KB


In [21]:
county_table.head()

,State,County,latitude,longitude
0,California,Alameda,37.646493,-121.887649
1,California,Alpine,38.597333,-119.820421
2,California,Amador,38.446065,-120.651756
3,California,Butte,39.667092,-121.600911
4,California,Cahuilla Indian Reservation,33.520240,-116.711760


## Table about Region

In [22]:
# need the county, State

region = research[['State', 'Region']]

In [23]:
region_table = region.groupby(['State', 'Region']).agg(counts = ('Region', 'size')).reset_index()

In [24]:
region_table.drop(columns=['counts'], inplace=True)

In [25]:
region_table.shape

(8, 2)

In [26]:
# the combine of each Region


# lookup = {
#     'Los Angeles CA' : ['Los Angeles CA']
#     'San Francisco-Oakland-San Jose CA' : ['Alameda', 'Contra Costa', 'Marin', 'Napa', 'San Francisco', 'San Mateo', 'Santa Clara', 'Solano', 'Sonoma']
#     'San Diego CA' : ['San Diego CA']
#     'Sacramento-Stockton-Modesto CA' : [Amador, CA
# }


### 1. County in the 'Sacramento-Stockton-Modesto CA' Region

In [27]:
# import the StrinIO function
# from io module
from io import StringIO
import pandas as pd
 
# wrap the string data in StringIO function
StringData = StringIO("""
County,State
Amador,California
Calaveras,California
Colusa,California
Nevada,California
Placer,California
Plumas,California
Sacramento,California
San Joaquin,California
Sierra,California
Stanislaus,California
Sutter,California
Tuolumne,California
Yolo,California
Yuba,California
    """)

# let's read the data using the Pandas
# read_csv() function
df = pd.read_csv(StringData, sep =",")
 
# Print the dataframe

In [28]:
df['Region'] = 'Sacramento-Stockton-Modesto CA'

#'Sacramento-Stockton-Modesto CA'

In [29]:
df

,County,State,Region
0,Amador,California,Sacramento-Stockton-Modesto CA
1,Calaveras,California,Sacramento-Stockton-Modesto CA
2,Colusa,California,Sacramento-Stockton-Modesto CA
3,Nevada,California,Sacramento-Stockton-Modesto CA
4,Placer,California,Sacramento-Stockton-Modesto CA
5,Plumas,California,Sacramento-Stockton-Modesto CA
6,Sacramento,California,Sacramento-Stockton-Modesto CA
7,San Joaquin,California,Sacramento-Stockton-Modesto CA
8,Sierra,California,Sacramento-Stockton-Modesto CA
9,Stanislaus,California,Sacramento-Stockton-Modesto CA


### 2. County in the 'Los_CA' Region - only Los Angeles County

### 3. County in the 'San Francisco-Oakland-San Jose CA' Region

In [30]:
a = ['Alameda', 'Contra Costa', 'Marin', 'Napa', 'San Francisco', 'San Mateo', 'Santa Clara', 'Solano', 'Sonoma']

df_sf = pd.DataFrame(a)

In [31]:
#'San Francisco-Oakland-San Jose CA'

In [32]:
df_sf['State'] = 'California'

In [33]:
df_sf['Region'] = 'San Francisco-Oakland-San Jose CA'

In [34]:
df_sf.rename(columns={0:'County'}, inplace = True)

In [35]:
df_sf

,County,State,Region
0,Alameda,California,San Francisco-Oakland-San Jose CA
1,Contra Costa,California,San Francisco-Oakland-San Jose CA
2,Marin,California,San Francisco-Oakland-San Jose CA
3,Napa,California,San Francisco-Oakland-San Jose CA
4,San Francisco,California,San Francisco-Oakland-San Jose CA
5,San Mateo,California,San Francisco-Oakland-San Jose CA
6,Santa Clara,California,San Francisco-Oakland-San Jose CA
7,Solano,California,San Francisco-Oakland-San Jose CA
8,Sonoma,California,San Francisco-Oakland-San Jose CA


### 4. County in the 'San Diego CA' Region - only San Diego County

### 5. County in the 'Austin' Region - only Austin

### 6. County in the 'SanAntonio' Region

In [36]:

 
# wrap the string data in StringIO function
StringData = StringIO("""
County,State
Atascosa,Texas
Bandera,Texas
Comal,Texas
Frio,Texas
Gillespie,Texas
Guadalupe,Texas
Karnes,Texas
Kendall,Texas
Kerr,Texas
Medina,Texas
Wilson,Texas
    """)

# let's read the data using the Pandas
# read_csv() function
df_sa = pd.read_csv(StringData, sep =",")
 
# Print the dataframe

In [37]:
df_sa['Region'] = 'SanAntonio'

In [38]:
df_sa

,County,State,Region
0,Atascosa,Texas,SanAntonio
1,Bandera,Texas,SanAntonio
2,Comal,Texas,SanAntonio
3,Frio,Texas,SanAntonio
4,Gillespie,Texas,SanAntonio
5,Guadalupe,Texas,SanAntonio
6,Karnes,Texas,SanAntonio
7,Kendall,Texas,SanAntonio
8,Kerr,Texas,SanAntonio
9,Medina,Texas,SanAntonio


### 7. County in the 'DFW' Region

In [39]:

 
# wrap the string data in StringIO function
StringData = StringIO("""
County,State
Dallas,Texas
Tarrant,Texas
Collin,Texas
Denton,Texas
Ellis,Texas
Johnson,Texas
Parker,Texas
Kaufman,Texas
Rockwall,Texas
Hunt,Texas
Wise,Texas
    """)

# let's read the data using the Pandas
# read_csv() function
df_dfw = pd.read_csv(StringData, sep =",")
 
# Print the dataframe

In [40]:
df_dfw['Region'] = 'DFW'

In [41]:
df_dfw

,County,State,Region
0,Dallas,Texas,DFW
1,Tarrant,Texas,DFW
2,Collin,Texas,DFW
3,Denton,Texas,DFW
4,Ellis,Texas,DFW
5,Johnson,Texas,DFW
6,Parker,Texas,DFW
7,Kaufman,Texas,DFW
8,Rockwall,Texas,DFW
9,Hunt,Texas,DFW


### Combine 7 tables

In [42]:
# 4 df

combine = pd.concat([df, df_sf, df_sa, df_dfw], join='outer', ignore_index=True)

In [43]:
# + 3 austin, la, San Diego CA
LA = {'County':'Los Angeles', 'State':'California', 'Region':'Los Angeles CA'}
Au = {'County':'Austin', 'State':'Texas', 'Region':'Austin'}
sd = {'County':'San Diego', 'State':'California', 'Region':'San Diego CA'}

combine.loc[len(combine)] = LA
combine.loc[len(combine)] = Au
combine.loc[len(combine)] = sd

In [44]:
combine.head()

,County,State,Region
0,Amador,California,Sacramento-Stockton-Modesto CA
1,Calaveras,California,Sacramento-Stockton-Modesto CA
2,Colusa,California,Sacramento-Stockton-Modesto CA
3,Nevada,California,Sacramento-Stockton-Modesto CA
4,Placer,California,Sacramento-Stockton-Modesto CA


In [45]:
# know what counties are in what region so that counties outside this table can match counties inside this table

combine.rename(columns={'County':'CountyInsideRegion'}, inplace=True)

In [46]:
combine.tail(10)

,CountyInsideRegion,State,Region
38,Ellis,Texas,DFW
39,Johnson,Texas,DFW
40,Parker,Texas,DFW
41,Kaufman,Texas,DFW
42,Rockwall,Texas,DFW
43,Hunt,Texas,DFW
44,Wise,Texas,DFW
45,Los Angeles,California,Los Angeles CA
46,Austin,Texas,Austin
47,San Diego,California,San Diego CA


In [47]:
combine.shape

(48, 3)

In [48]:
# find la and long for combine table

combine['latitude'] = 0.0
combine['longitude'] = 0.0

In [49]:
for idx in range(len(combine)):
    try:
        add = combine.iloc[idx, 0] + 'county' + combine.iloc[idx, 1]
        cur_la_lo = list(nom.geocode(add))[1]
        combine.iloc[idx, 3] = cur_la_lo[0]
        combine.iloc[idx, 4] = cur_la_lo[1]
    except:
        print('Something wrong with the idx:' + str(idx))

In [50]:
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48 entries, 0 to 47
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   CountyInsideRegion  48 non-null     object 
 1   State               48 non-null     object 
 2   Region              48 non-null     object 
 3   latitude            48 non-null     float64
 4   longitude           48 non-null     float64
dtypes: float64(2), object(3)
memory usage: 2.2+ KB


In [51]:
combine.head()

,CountyInsideRegion,State,Region,latitude,longitude
0,Amador,California,Sacramento-Stockton-Modesto CA,38.446065,-120.651756
1,Calaveras,California,Sacramento-Stockton-Modesto CA,38.204736,-120.554065
2,Colusa,California,Sacramento-Stockton-Modesto CA,39.177598,-122.237103
3,Nevada,California,Sacramento-Stockton-Modesto CA,33.663954,-93.307114
4,Placer,California,Sacramento-Stockton-Modesto CA,39.059136,-120.753631


In [52]:
#county_table.drop(columns=['countyInsideRegion', 'low_dis', 'CountyInsideRegion'], inplace=True)

In [53]:
import math
import collections
county_table['CountyInsideRegion'] = ''
county_table['low_dis'] = 0

In [54]:
county_table.head()

,State,County,latitude,longitude,CountyInsideRegion,low_dis
0,California,Alameda,37.646493,-121.887649,,0
1,California,Alpine,38.597333,-119.820421,,0
2,California,Amador,38.446065,-120.651756,,0
3,California,Butte,39.667092,-121.600911,,0
4,California,Cahuilla Indian Reservation,33.520240,-116.711760,,0


In [55]:
M = len(county_table)
N = len(combine)

for i in range(M): # county in table ca
    x1 = county_table.iloc[i,2]
    x2 = county_table.iloc[i,3]
    min_dis = float('inf')

    for idx in range(N): # combine county region table
        y1 = combine.iloc[idx,3]
        y2 = combine.iloc[idx,4]

        dis = math.dist([x1,x2],[y1,y2])
        #lookup[combine_ca.iloc[idx,0]] = dis

        if dis < min_dis and dis != 0:
            min_dis = dis
            countyinsideregion = combine.iloc[idx,0]
            
    county_table.iloc[i,4] = countyinsideregion
    county_table.iloc[i,5] = min_dis


In [56]:
county_table.head()

,State,County,latitude,longitude,CountyInsideRegion,low_dis
0,California,Alameda,37.646493,-121.887649,Contra Costa,0.274131
1,California,Alpine,38.597333,-119.820421,Tuolumne,0.585290
2,California,Amador,38.446065,-120.651756,Calaveras,0.260352
3,California,Butte,39.667092,-121.600911,Yuba,0.469739
4,California,Cahuilla Indian Reservation,33.520240,-116.711760,San Diego,0.488387


In [57]:
county_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 320 entries, 0 to 319
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               320 non-null    object 
 1   County              320 non-null    object 
 2   latitude            320 non-null    float64
 3   longitude           320 non-null    float64
 4   CountyInsideRegion  320 non-null    object 
 5   low_dis             320 non-null    float64
dtypes: float64(3), object(3)
memory usage: 15.1+ KB


### Match county and region

In [58]:
t1 = county_table.copy()

In [59]:
t1.head()

,State,County,latitude,longitude,CountyInsideRegion,low_dis
0,California,Alameda,37.646493,-121.887649,Contra Costa,0.274131
1,California,Alpine,38.597333,-119.820421,Tuolumne,0.585290
2,California,Amador,38.446065,-120.651756,Calaveras,0.260352
3,California,Butte,39.667092,-121.600911,Yuba,0.469739
4,California,Cahuilla Indian Reservation,33.520240,-116.711760,San Diego,0.488387


In [60]:
t1.drop(columns=['latitude', 'longitude', 'low_dis'], inplace=True)

In [61]:
t1.shape

(320, 3)

In [62]:
t1[t1.County == 'Nevada']

,State,County,CountyInsideRegion
31,California,Nevada,Hunt


#### t1 is like the dictionary to find what counties are in the region

In [63]:
#t2 = region_table.copy()

In [64]:
#t2.head()

In [66]:
t3 = combine.copy()

t3.drop(columns=['latitude', 'longitude'], inplace=True )

In [67]:
t3.head()

,CountyInsideRegion,State,Region
0,Amador,California,Sacramento-Stockton-Modesto CA
1,Calaveras,California,Sacramento-Stockton-Modesto CA
2,Colusa,California,Sacramento-Stockton-Modesto CA
3,Nevada,California,Sacramento-Stockton-Modesto CA
4,Placer,California,Sacramento-Stockton-Modesto CA


#### match region in t3 to t1 so that we can match search for the next step

In [68]:
t4 = t1.merge(t3, on=['CountyInsideRegion', 'State'], how='inner')

In [69]:
t4.isna().sum()

State                 0
County                0
CountyInsideRegion    0
Region                0
dtype: int64

In [70]:
t4.head()

,State,County,CountyInsideRegion,Region
0,California,Alameda,Contra Costa,San Francisco-Oakland-San Jose CA
1,California,Solano,Contra Costa,San Francisco-Oakland-San Jose CA
2,California,Alpine,Tuolumne,Sacramento-Stockton-Modesto CA
3,California,Fresno,Tuolumne,Sacramento-Stockton-Modesto CA
4,California,Madera,Tuolumne,Sacramento-Stockton-Modesto CA


In [71]:
t4[t4.County == 'Trinity']

,State,County,CountyInsideRegion,Region
245,Texas,Trinity,Austin,Austin


In [72]:
t4.shape

(311, 4)

In [73]:
problem = t4[t4.Region.isna()].County.tolist()

In [74]:
for i in problem:
    if i in t1.County.unique():
        print(i + " is in t1.County.unique")
        
    if i in t3.CountyInsideRegion.unique():
        print(i + " is in t3.CountyInsideRegion.unique")

In [75]:
t4.head()

,State,County,CountyInsideRegion,Region
0,California,Alameda,Contra Costa,San Francisco-Oakland-San Jose CA
1,California,Solano,Contra Costa,San Francisco-Oakland-San Jose CA
2,California,Alpine,Tuolumne,Sacramento-Stockton-Modesto CA
3,California,Fresno,Tuolumne,Sacramento-Stockton-Modesto CA
4,California,Madera,Tuolumne,Sacramento-Stockton-Modesto CA


In [76]:
t4[t4.County == 'Houston']

,State,County,CountyInsideRegion,Region
272,Texas,Houston,Johnson,DFW


### Match ca_de and te_de with county_table by key State and County

In [77]:

# read density table
ca_de = pd.read_excel('/Users/chensumei/Desktop/Ram Paper/FinalSheets/Population_Density_CA_TE.xlsx', sheet_name='California')
te_de = pd.read_excel('/Users/chensumei/Desktop/Ram Paper/FinalSheets/Population_Density_CA_TE.xlsx', sheet_name='Texas')
combine_De = ca_de.merge(te_de, how = 'outer')

In [78]:
combine_De.head()

,Population Density,County,State,Population
0,3575.3,San Francisco,California,829072
1,3255.4,Orange,California,3086331
2,2099.4,Los Angeles,California,9974203
3,1898.5,Alameda,California,1559308
4,1459.0,Sacramento,California,1450277


In [79]:
combine_De.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 312 entries, 0 to 311
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Population Density  312 non-null    float64
 1   County              312 non-null    object 
 2   State               312 non-null    object 
 3   Population          312 non-null    int64  
dtypes: float64(1), int64(1), object(2)
memory usage: 12.2+ KB


In [80]:
combine_De[combine_De.County == 'De Witt']

,Population Density,County,State,Population


In [81]:

combine_De[combine_De.County == 'Kickapoo Reservation']

,Population Density,County,State,Population


In [82]:
#combine density and t4

test = t4.merge(combine_De, on=['State', 'County'], how='inner')

In [83]:
test.isna().sum()

State                 0
County                0
CountyInsideRegion    0
Region                0
Population Density    0
Population            0
dtype: int64

In [84]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 303 entries, 0 to 302
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   State               303 non-null    object 
 1   County              303 non-null    object 
 2   CountyInsideRegion  303 non-null    object 
 3   Region              303 non-null    object 
 4   Population Density  303 non-null    float64
 5   Population          303 non-null    int64  
dtypes: float64(1), int64(1), object(4)
memory usage: 16.6+ KB


In [85]:
test[test.County == 'Houston']

,State,County,CountyInsideRegion,Region,Population Density,Population
264,Texas,Houston,Johnson,DFW,17.9,22066


In [86]:
idx = []

for i in range(len(test)):
    if test.iloc[i, 1] == 'Houston':
        idx.append(i)

In [88]:
idx

[264]

In [87]:
test.iloc[264,3] = 'Houston'

## begin to merge google and final table

In [93]:
final = research.merge(test, on=['State', 'Region'], how='outer')

In [94]:
others.head()

,County,SNAP_Applications,Year,Month,State,Number of Workers,# of disaster
0,Anderson,405,2019,1,Texas,0,0
1,Anderson,243,2019,2,Texas,0,0
2,Anderson,282,2019,3,Texas,0,0
3,Anderson,274,2019,4,Texas,0,0
4,Anderson,317,2019,5,Texas,0,0


In [95]:
final.head()

,Search,Search term,Region,State,Year,Month,County,CountyInsideRegion,Population Density,Population
0,24,calfresh,Los Angeles CA,California,2019,1,Inyo,Los Angeles,1.8,18439
1,24,calfresh,Los Angeles CA,California,2019,1,Kern,Los Angeles,105.1,857730
2,24,calfresh,Los Angeles CA,California,2019,1,Orange,Los Angeles,3255.4,3086331
3,24,calfresh,Los Angeles CA,California,2019,1,San Diego,Los Angeles,703.4,3183143
4,24,calfresh,Los Angeles CA,California,2019,1,Santa Barbara,Los Angeles,113.9,431555


In [105]:
df = final.merge(others,on=['State', 'County', 'Year', 'Month'], how='left')

In [106]:
df.head()

,Search,Search term,Region,State,Year,Month,County,CountyInsideRegion,Population Density,Population,SNAP_Applications,Number of Workers,# of disaster
0,24,calfresh,Los Angeles CA,California,2019,1,Inyo,Los Angeles,1.8,18439,84.0,0.0,0.0
1,24,calfresh,Los Angeles CA,California,2019,1,Kern,Los Angeles,105.1,857730,6163.0,134.0,0.0
2,24,calfresh,Los Angeles CA,California,2019,1,Orange,Los Angeles,3255.4,3086331,8637.0,200.0,0.0
3,24,calfresh,Los Angeles CA,California,2019,1,San Diego,Los Angeles,703.4,3183143,12905.0,140.0,0.0
4,24,calfresh,Los Angeles CA,California,2019,1,Santa Barbara,Los Angeles,113.9,431555,2509.0,0.0,0.0


In [107]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 236340 entries, 0 to 236339
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Search              236340 non-null  int64  
 1   Search term         236340 non-null  object 
 2   Region              236340 non-null  object 
 3   State               236340 non-null  object 
 4   Year                236340 non-null  int64  
 5   Month               236340 non-null  int64  
 6   County              236340 non-null  object 
 7   CountyInsideRegion  236340 non-null  object 
 8   Population Density  236340 non-null  float64
 9   Population          236340 non-null  int64  
 10  SNAP_Applications   235580 non-null  float64
 11  Number of Workers   235580 non-null  float64
 12  # of disaster       235580 non-null  float64
dtypes: float64(4), int64(4), object(5)
memory usage: 25.2+ MB


In [111]:
df.isna().sum()

Search                0
Search term           0
Region                0
State                 0
Year                  0
Month                 0
County                0
CountyInsideRegion    0
Population Density    0
Population            0
SNAP_Applications     0
Number of Workers     0
# of disaster         0
dtype: int64

In [110]:
df.fillna(0, inplace=True)

In [112]:
df.to_excel('final_database.xlsx')